# Final Assignment for Applied Data Science Capstone:

---

#### This notebook is one of the final assignment of the Data Science Specialization. 9-Courses created by IBM on Coursera Platform.

#### The Battle of the Neighborhoods

**According to the instruction ,**

**" you will have the opportunity to be as creative as you want and come up with an idea**

**to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice**

**or to come up with a problem that you can use the Foursquare location data to solve ".**

## Summary

**Basically, what we need to do is try to apply data science technique to location data with a little help from 4SquareAPI we managed to get venues nearby the neighborhoods**
**For this location dataset I chooses Bangkok because I studied in Bangkok for 4 years and I still know tiny bit of it because Bangkok is so huge. it's divided into 50 districts and lots of sub-district .**

**Main Objectives:**
* Explore Bangkok how place are located in Bangkok
 - Is the innercore side of bangkok have more tourist attraction?


* If we are looking for an oppotunity to open a Bar in Bangkok, where should we landed?
 - Where we wanted to land is a place that we have enough customer but don't have a lot of bar around

---

### Import Neccessaries Packages

In [58]:
# packages for manipulating df
import numpy as np
import pandas as pd
# packages for read_html wikipage
from pandas.io.html import read_html
import matplotlib.cm as cm
import matplotlib.colors as colors
#slkearn for KMeans Cluster
from sklearn.cluster import KMeans
import json
import requests
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from pandas.io.json import json_normalize

Solving environment: done

# All requested packages already installed.



### Bangkok Location Data contain district, sub-district, postalcode, latitude, longitude from Wikipages.

#### - Assigned wiki url to pages1, pages2, pages3

In [4]:
pages1 = 'https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok' # BKK: district, lat, long
pages2 = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%A3%E0%B8%AB%E0%B8%B1%E0%B8%AA%E0%B9%84%E0%B8%9B%E0%B8%A3%E0%B8%A9%E0%B8%93%E0%B8%B5%E0%B8%A2%E0%B9%8C%E0%B9%84%E0%B8%97%E0%B8%A2'
# BKK: district, postalcode
pages3 = 'https://th.wikipedia.org/wiki/%E0%B9%81%E0%B8%82%E0%B8%A7%E0%B8%87' # BKK: district, sub-district

---

### Editting, Transforming, Loading Data :ETL

In [5]:
# Read html from pages1, set index columns = 0, class wikitable and assigned to table1
table1 = read_html(pages1, index_col = 0, attrs = {'class':'wikitable'})
# Reset index from table1[0] and assigned to df1
df1 = table1[0].reset_index()
# For df1 drop cols = ['Code', 'Population', 'No.ofSubdistricts(Khwaeng)']
df1.drop(['Population', 'No. ofSubdistricts(Khwaeng)'], axis = 1, inplace = True)
# For df1 set index to be cols = ['Thai']
df1.set_index('Thai', inplace = True)

In [6]:
# Create a dictionary contain district Code of Phra Nakhon assigned to Pra_Nakhon
Phra_Nakhon = {1: 'Phra Nakhon', 2: 'Phra Nakhon', 29: 'Phra Nakhon', 30: 'Phra Nakhon', 14: 'Phra Nakhon', 37: 'Phra Nakhon', 7: 'Phra Nakhon', 31: 'Phra Nakhon', 12: 'Phra Nakhon', 8: 'Phra Nakhon', 13: 'Phra Nakhon', 4: 'Phra Nakhon', 33: 'Phra Nakhon', 39: 'Phra Nakhon', 26: 'Phra Nakhon', 17: 'Phra Nakhon', 34: 'Phra Nakhon', 9: 'Phra Nakhon', 47: 'Phra Nakhon', 32: 'Phra Nakhon', 45: 'Phra Nakhon', 6: 'Phra Nakhon', 38: 'Phra Nakhon', 41: 'Phra Nakhon', 5: 'Phra Nakhon', 27: 'Phra Nakhon', 44: 'Phra Nakhon', 43: 'Phra Nakhon', 36: 'Phra Nakhon', 42: 'Phra Nakhon', 46: 'Phra Nakhon', 10: 'Phra Nakhon', 11: 'Phra Nakhon', 3: 'Phra Nakhon', 28: 'Phra Nakhon'}
# Matches the df1['Code'] to Phra_Nakhon
df1['Code'] = df1['Code'].map(Phra_Nakhon)
#fill other Code with 'Thorn Buri'
df1['Code'].fillna('Thorn Buri', inplace = True)
df1.head()

,District(Khet),Code,Latitude,Longitude
Thai,,,,
บางบอน,Bang Bon,Thorn Buri,NaN,NaN
บางกะปิ,Bang Kapi,Phra Nakhon,13.765833,100.647778
บางแค,Bang Khae,Thorn Buri,13.696111,100.409444
บางเขน,Bang Khen,Phra Nakhon,13.873889,100.596389
บางคอแหลม,Bang Kho Laem,Phra Nakhon,13.693333,100.502500


In [7]:
# Replace missing value with list of value that we retreived from latitude, longitude geocode.
df1['Latitude'].fillna(13.6665033, inplace = True, limit = 1)
df1['Longitude'].fillna(100.4288593, inplace = True, limit = 1)
df1['Latitude'].fillna(13.8188814, inplace = True, limit = 1)
df1['Longitude'].fillna(100.678848325432, inplace = True, limit = 1)
df1['Latitude'].fillna(13.7726302, inplace = True, limit = 1)
df1['Longitude'].fillna(100.3535049, inplace = True, limit = 1)
df1['Latitude'].fillna(13.62542025, inplace = True, limit = 1)
df1['Longitude'].fillna(100.49378289535, inplace = True, limit = 1)
df1['Latitude'].fillna(13.7542529, inplace = True, limit = 1)
df1['Longitude'].fillna(100.493087, inplace = True, limit = 1)

In [8]:
print('This DataFrame has {} rows and columns respectively.'.format(df1.shape))
df1.head()

This DataFrame has (50, 4) rows and columns respectively.


,District(Khet),Code,Latitude,Longitude
Thai,,,,
บางบอน,Bang Bon,Thorn Buri,13.666503,100.428859
บางกะปิ,Bang Kapi,Phra Nakhon,13.765833,100.647778
บางแค,Bang Khae,Thorn Buri,13.696111,100.409444
บางเขน,Bang Khen,Phra Nakhon,13.873889,100.596389
บางคอแหลม,Bang Kho Laem,Phra Nakhon,13.693333,100.502500


In [9]:
# Read html from pages2, set index columns = 0, class wikitable and assigned to table2
table2 = read_html(pages2, index_col = 0, attrs = {"class":"wikitable"})
# Reset index from table1[0] and assigned to df2
df2 = table2[0].reset_index()
# For df2 drop cols = ['หมายเหตุ']
df2.drop(['หมายเหตุ'], axis = 1, inplace = True)
# For df2 set index to be cols = ['Thai']
df2.set_index('เขต', inplace = True)
print('This DataFrame has {} rows and columns respectively.'.format(df2.shape))
df2.head()

This DataFrame has (50, 1) rows and columns respectively.


,รหัสไปรษณีย์
เขต,
คลองเตย,10110
คลองสาน,10600
คลองสามวา,10510
คันนายาว,10230
จตุจักร,10900


In [10]:
# Read html from pages3, set index columns = 0, class wikitable and assigned to table3
table3 = read_html(pages3, index_col = 0, attrs = {"class":"wikitable"})
# Reset index from table3[0] and assigned to a
a = table3[0].reset_index()
# Export a to csv, not including index by using index = False
a.to_csv('df3.csv', index = False)
# Using pandas .read_csv read 'df3.csv' not including header by using header = None and assigned to df3
df3 = pd.read_csv('df3.csv', header = None)
# For df3 drop cols = [0, 2, 4] and rows = ['0', 1]
df3.drop([0, 2, 4], axis = 1, inplace = True)
df3.drop([0, 1], axis = 0, inplace = True)
# For df3 grouped by cols = [1] and join cols = [3] using ',' for separate and assigned back to df3
df3 = df3.groupby(1, as_index = False).agg(lambda x: ',' .join(x))
# For df3 set index to be cols = [1]
df3.set_index(1, inplace = True)
print('This DataFrame has {} rows and columns respectively.'.format(df3.shape))
df3.head()

This DataFrame has (50, 1) rows and columns respectively.


,3
1,
คลองสาน,"สมเด็จเจ้าพระยา,คลองสาน,บางลำภูล่าง,คลองต้นไทร"
คลองสามวา,"สามวาตะวันตก,สามวาตะวันออก,บางชัน,ทรายกองดิน,ท..."
คลองเตย,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห..."
คันนายาว,"คันนายาว,รามอินทรา"
จตุจักร,"ลาดยาว,เสนานิคม,จันทรเกษม,จอมพล,จตุจักร"


In [11]:
# Merged df1, df2, df3 join inner ['Thai', 'เขต', '1'] by using pandas .concat()
Merged_df = pd.concat([df1, df2, df3], axis = 1, join = 'inner')
Merged_df.reset_index()
# For Merged_df rename cols = ['index', 'District(Khet)', 'รหัสไปรษณีย์', 3]
Merged_df.rename(columns = {'index':'เขต', 'District(Khet)':'District', 'รหัสไปรษณีย์':'PostalCode', 3:'Sub-district', 'Code':'Side'}, inplace = True)
# Export Merged_df to csv, not including index by using index = False
Merged_df.to_csv('bkklocation.csv', index = False)
# Using pandas .read_csv read 'bkk_location'.csv and assigned to df 
df = pd.read_csv('bkklocation.csv')
print('This DataFrame has {} rows and columns respectively.'.format(df.shape))
df.head()

This DataFrame has (50, 6) rows and columns respectively.


,District,Side,Latitude,Longitude,PostalCode,Sub-district
0,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
1,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง..."
2,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง"
3,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย..."
4,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่"


In [12]:
df.to_csv('df.csv', index = False)
bkk_location = pd.read_csv('df.csv')
bkk_location.reset_index()
print(bkk_location.shape)
bkk_location.head()

(50, 6)


,District,Side,Latitude,Longitude,PostalCode,Sub-district
0,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
1,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง..."
2,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง"
3,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย..."
4,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่"


#### Bangkok is divided by the Chao Phraya River, The side which have Royal Palace Called 'Phra Nakhon' and the other side called 'Thorn Buri' .

In [16]:
#Endcoded df1['Code'] which contain Side of Bangkok with one hot endcode using pandas .get_dummies and assigned to endcode
endcode = pd.get_dummies(df1[['Code']], prefix = "", prefix_sep = "")
endcode['Code'] = df1['Code']
endcode.head()

,Phra Nakhon,Thorn Buri,Code
Thai,,,
บางบอน,0,1,Thorn Buri
บางกะปิ,1,0,Phra Nakhon
บางแค,0,1,Thorn Buri
บางเขน,1,0,Phra Nakhon
บางคอแหลม,1,0,Phra Nakhon


#### Now we have bkk_location which contain location data that we need about Bangkok districts. <br><br>For now we going to address ourselve in 'Bangkok, Bangkhen' and retreived its coordinates by using geocode packages.

In [17]:
#address for bkk Bangkhen
address = 'Bangkok, Bangkhen'

geolocator = Nominatim(user_agent="Bkk_neigh-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Bangkok, Bangkhen are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Bangkok, Bangkhen are 13.8195527, 100.5133844.


In [14]:
# create bkk map using folium
BKK_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map assigned i, j, k, l to bkk_location['Latitude', 'Longitude', 'District', 'Sub-district']
for i, j, k, l, in zip(bkk_location['Latitude'], bkk_location['Longitude'], bkk_location['District'], bkk_location['Sub-district']):
    label = '{}, {}'.format(l, k)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [i, j], #latitude, #longitude
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(BKK_map)  
    
BKK_map

#### This Visualization shows before the cluster of Bangkok based on their Side of the river.

In [18]:
Bkk_map = folium.Map(location=[latitude, longitude], zoom_start=12)

X = endcode['Phra Nakhon']
Y = endcode['Thorn Buri']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=2, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'blue']
bkk_location['Cluster'] = clusters

for latitude, longitude, Side, cluster, District in zip(bkk_location['Latitude'], bkk_location['Longitude'], bkk_location['Side'], bkk_location['Cluster'], bkk_location['District']):
    label = '{} {}'.format(Side, District)
    label = folium.Popup(str(Side) + ' , ' + District, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(Bkk_map)  

Bkk_map

#### This Visualization shows cluster of Bangkok based on their Side of the river red is 'Thorn Buri Side', blue is 'Phra Nakhon Side' .

### Using FOURSQUARE API to get venues.

In [65]:
#Foursquare credentials
CLIENT_ID = 'VFQIALTA3BOXJMGBWZ1JAHCTOE5W1FOJ04V1Y54JS0E5E5TD' # your Foursquare ID
CLIENT_SECRET = 'XNYUP24ROZRLUKQRZRURJWPUOZQV3HXV3FKO2QAFW2UZYDU1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('loading........')

Your credentails:
CLIENT_ID: VFQIALTA3BOXJMGBWZ1JAHCTOE5W1FOJ04V1Y54JS0E5E5TD
CLIENT_SECRET:XNYUP24ROZRLUKQRZRURJWPUOZQV3HXV3FKO2QAFW2UZYDU1
loading........


In [66]:
LIMIT = 1000
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [104]:
bkk_venues = getNearbyVenues(names=bkk_location['Sub-district'],
                                   latitudes=bkk_location['Latitude'],
                                   longitudes=bkk_location['Longitude']
                                  )

บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบางบอน
คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก
บางแค,บางแคเหนือ,บางไผ่,หลักสอง
ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สายไหม,ทุ่งสองห้อง,ท่าแร้ง,ออเงิน
บางคอแหลม,วัดพระยาไกร,บางโคล่
บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด,แสมดำ
บางนา,บางนาเหนือ,บางนาใต้
บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน
มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา
บางซื่อ,วงศ์สว่าง
บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บางขุนนนท์,บางขุนศรี,บางยี่ขัน,อรุณอมรินทร์
วัดอรุณ,วัดท่าพระ
คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์
ลาดยาว,เสนานิคม,จันทรเกษม,จอมพล,จตุจักร
บางขุนเทียน,บางค้อ,บางมด,จอมทอง
ดินแดง,รัชดาภิเษก
ตลาดบางเขน,สีกัน,ทุ่งสองห้อง,ดอนเมือง,สนามบิน
ดุสิต,วชิรพยาบาล,สวนจิตรลดา,สี่แยกมหานาค,บางซื่อ,ถนนนครไชยศรี
ห้วยขวาง,บางกะปิ,ดินแดง,สามเสนนอก
คันนายาว,รามอินทรา
สามวาตะวันตก,สามวาตะวันออก,บางชัน,ทรายกองดิน,ทรายกองดินใต้
สมเด็จเจ้าพระยา,คลองสาน,บางลำภูล่าง,คลองต้นไทร
คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระ

In [105]:
print(bkk_venues.shape)
bkk_venues.head()

(2882, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,Shabu Na-Mor (ชาบูหน้าหม้อ),13.663855,100.428756,Shabu-Shabu Restaurant
1,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ก๋วยเตี๋ยวตำลึง นายเอก,13.667102,100.428354,Noodle House
2,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ส้มตำครกเงิน,13.666023,100.432755,Som Tum Restaurant
3,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ไผ่ทองโภชนา,13.662101,100.435264,Thai Restaurant
4,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ขนมจีนเทวดา บีบเส้นสดๆ,13.659428,100.433692,Noodle House


In [106]:
#There are 164 uniques venues categories from cols = ['Venue Category'] 
print('There are {} uniques venues categories.'.format(len(bkk_venues['Venue Category'].unique())))
#Group Bangkok_venues by cols =['Neighborhood'] and count all of the cols 
bkk_venues.groupby('Neighborhood').count().head()

There are 233 uniques venues categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง",19,19,19,19,19,19
"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",18,18,18,18,18,18
"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก",100,100,100,100,100,100
"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวีวัฒนา,บางเชือกหนัง,ศาลาธรรมสพน์",42,42,42,42,42,42
"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",23,23,23,23,23,23


In [107]:
bkk_venues['Venue Category'].value_counts()

Thai Restaurant              246
Noodle House                 244
Coffee Shop                  183
Convenience Store            141
Café                         122
Asian Restaurant              98
Hotel                         83
Japanese Restaurant           83
Chinese Restaurant            79
Bar                           67
Restaurant                    54
Dessert Shop                  53
Seafood Restaurant            48
Bakery                        44
Fast Food Restaurant          43
Som Tum Restaurant            41
Steakhouse                    40
Hotpot Restaurant             39
Shopping Mall                 38
Ice Cream Shop                36
BBQ Joint                     36
Hostel                        31
Supermarket                   30
Spa                           28
Pizza Place                   26
Shabu-Shabu Restaurant        25
Gym / Fitness Center          25
Market                        24
Flea Market                   22
Pub                           22
          

In [108]:
# one hot encoding
bkk_endcode = pd.get_dummies(bkk_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood cols
bkk_endcode['Neighborhood'] = bkk_venues['Neighborhood']
bkk_endcode.head()

,Accessories Store,African Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Water Park,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
cols_list = bkk_endcode.columns.tolist()
# move Neighborhood to index 0 of cols_list
cols_list.insert(0, cols_list.pop(cols_list.index('Neighborhood')))
cols_list
# Reindex cols in bkk_endcode by assign to cols_list and assign it back to bkk_endcode
bkk_endcode = bkk_endcode.reindex(columns = cols_list)

bkk_grouped = bkk_endcode.groupby('Neighborhood').mean().reset_index().round(2)
print(bkk_grouped.shape)
bkk_grouped.head()

(50, 233)


,Neighborhood,Accessories Store,African Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Water Park,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0
1,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.06,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0
2,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.00,...,0.0,0.0,0.01,0.0,0.01,0.01,0.0,0.01,0.0,0.0
3,"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวี...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0
4,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.00,...,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0


In [110]:
# Assign 5 to num_to_venues 
num_top_venues = 5
# Create function that create temp that contain venue and freq of each hood in neighborhood 
for hood in bkk_grouped['Neighborhood']:
    print("----"+hood+"----") #
    temp = bkk_grouped[bkk_grouped['Neighborhood'] == hood].T.reset_index() #reset index
    temp.columns = ['venue','freq'] # create 2 temp col (venue, freq)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง----
               venue  freq
0        Coffee Shop  0.16
1      Shopping Mall  0.11
2         Restaurant  0.05
3        Supermarket  0.05
4  Convenience Store  0.05


----คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์----
               venue  freq
0        Coffee Shop  0.17
1  Convenience Store  0.11
2        Supermarket  0.11
3               Café  0.11
4    Thai Restaurant  0.06


----คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก----
                  venue  freq
0           Coffee Shop  0.08
1       Thai Restaurant  0.08
2   Japanese Restaurant  0.07
3  Fast Food Restaurant  0.05
4        Clothing Store  0.04


----คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวีวัฒนา,บางเชือกหนัง,ศาลาธรรมสพน์----
               venue  freq
0       Noodle House  0.14
1  Convenience Store  0.14
2        Coffee Shop  0.07
3               Café  0.07
4    Thai Restaurant  0.07


----คลองเตย,คลองตัน,

In [111]:
# Create a function that return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bkk_grouped['Neighborhood']

for ind in np.arange(bkk_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bkk_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",Coffee Shop,Shopping Mall,Gym / Fitness Center,Big Box Store,Thai Restaurant
1,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",Coffee Shop,Café,Supermarket,Convenience Store,Park
2,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Steakhouse
3,"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวี...",Noodle House,Convenience Store,Café,Coffee Shop,Thai Restaurant
4,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",Coffee Shop,Convenience Store,Thai Restaurant,Noodle House,Bar


In [114]:
# k = 5 
k = 5
# Drop col Neighborhood and assign to cluster
cluster = bkk_grouped.drop('Neighborhood', axis = 1)
# k-means Clustering
clustering = KMeans(n_clusters = k, random_state = 1).fit(cluster)
#check cluster label
clustering.labels_[0:10]

array([1, 1, 0, 2, 1, 1, 0, 0, 0, 2], dtype=int32)

In [115]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', clustering.labels_)
merged_group = bkk_location
#join
merged_group = merged_group.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on= 'Sub-district')
merged_group.head()

,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",0,0,Thai Restaurant,Bakery,Noodle House,Restaurant,Asian Restaurant
1,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",1,0,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Steakhouse
2,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง",0,2,Noodle House,Convenience Store,Coffee Shop,Japanese Restaurant,Asian Restaurant
3,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย...",1,0,Fast Food Restaurant,Asian Restaurant,Coffee Shop,Convenience Store,Noodle House
4,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่",1,1,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Seafood Restaurant


In [116]:
merged_group['Cluster Labels'].value_counts()

0    22
2    14
1    12
4     1
3     1
Name: Cluster Labels, dtype: int64

In [117]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for a, b, c, d, e, f in zip(merged_group['Latitude'], merged_group['Longitude'], merged_group['PostalCode'], merged_group['District'], merged_group['Sub-district'], merged_group['Cluster Labels']):
    label = folium.Popup(str(d) + ' Cluster ' + str(f), parse_html=True)
    folium.CircleMarker(
        [a, b],
        radius=5,
        popup=label,
        color=rainbow[f-1],
        fill=True,
        fill_color=rainbow[f-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [118]:
cluster0 = merged_group.loc[merged_group['Cluster Labels'] == 0, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster0.shape)
cluster0

(22, 14)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Thorn Buri,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",0,0,Thai Restaurant,Bakery,Noodle House,Restaurant,Asian Restaurant
1,Phra Nakhon,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",1,0,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Steakhouse
3,Phra Nakhon,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย...",1,0,Fast Food Restaurant,Asian Restaurant,Coffee Shop,Convenience Store,Noodle House
5,Thorn Buri,Bang Khun Thian,Thorn Buri,13.660833,100.435833,10150,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",0,0,Thai Restaurant,Coffee Shop,Restaurant,Japanese Restaurant,Ice Cream Shop
8,Phra Nakhon,Bang Rak,Phra Nakhon,13.730833,100.524167,10500,"มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา",1,0,Noodle House,Thai Restaurant,Hotel,Chinese Restaurant,Café
13,Phra Nakhon,Chatuchak,Phra Nakhon,13.828611,100.559722,10900,"ลาดยาว,เสนานิคม,จันทรเกษม,จอมพล,จตุจักร",1,0,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Hotel
18,Phra Nakhon,Huai Khwang,Phra Nakhon,13.776667,100.579444,10310,"ห้วยขวาง,บางกะปิ,ดินแดง,สามเสนนอก",1,0,Thai Restaurant,Noodle House,Japanese Restaurant,Hotel,Asian Restaurant
19,Phra Nakhon,Khan Na Yao,Phra Nakhon,13.818881,100.678848,10230,"คันนายาว,รามอินทรา",1,0,Thai Restaurant,Japanese Restaurant,Coffee Shop,Som Tum Restaurant,Café
20,Phra Nakhon,Khlong Sam Wa,Phra Nakhon,13.859722,100.704167,10510,"สามวาตะวันตก,สามวาตะวันออก,บางชัน,ทรายกองดิน,ท...",1,0,Thai Restaurant,Restaurant,Exhibit,Noodle House,Coffee Shop
21,Thorn Buri,Khlong San,Thorn Buri,13.730278,100.509722,10600,"สมเด็จเจ้าพระยา,คลองสาน,บางลำภูล่าง,คลองต้นไทร",0,0,Chinese Restaurant,Noodle House,Dessert Shop,Thai Restaurant,Coffee Shop


In [119]:
cluster1 = merged_group.loc[merged_group['Cluster Labels'] == 1, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster1.shape)
cluster1

(12, 14)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Phra Nakhon,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่",1,1,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Seafood Restaurant
7,Thorn Buri,Bang Phlat,Thorn Buri,13.793889,100.505000,10700,"บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน",0,1,Convenience Store,Thai Restaurant,Café,Fast Food Restaurant,Ice Cream Shop
12,Phra Nakhon,Bueng Kum,Phra Nakhon,13.785278,100.669167,10240,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",1,1,Coffee Shop,Café,Supermarket,Convenience Store,Park
14,Thorn Buri,Chom Thong,Thorn Buri,13.677222,100.484722,10150,"บางขุนเทียน,บางค้อ,บางมด,จอมทอง",0,1,Thai Restaurant,Convenience Store,Pet Store,Coffee Shop,Food Truck
16,Phra Nakhon,Don Mueang,Phra Nakhon,13.913611,100.589722,10210,"ตลาดบางเขน,สีกัน,ทุ่งสองห้อง,ดอนเมือง,สนามบิน",1,1,Convenience Store,Noodle House,Food Truck,Grocery Store,Thai Restaurant
22,Phra Nakhon,Khlong Toei,Phra Nakhon,13.708056,100.583889,10110,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",1,1,Coffee Shop,Convenience Store,Thai Restaurant,Noodle House,Bar
23,Phra Nakhon,Lak Si,Phra Nakhon,13.887500,100.578889,10210,"ทุ่งสองห้อง,ตลาดบางเขน",1,1,Convenience Store,Thai Restaurant,Coffee Shop,Japanese Restaurant,Steakhouse
26,Phra Nakhon,Min Buri,Phra Nakhon,13.813889,100.748056,10510,"มีนบุรี,แสนแสบ,บางชัน,ทรายกองดิน,ทรายกองดินใต้...",1,1,Intersection,Coffee Shop,Bakery,Massage Studio,Farm
27,Phra Nakhon,Nong Chok,Phra Nakhon,13.855556,100.862500,10530,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",1,1,Coffee Shop,Shopping Mall,Gym / Fitness Center,Big Box Store,Thai Restaurant
28,Thorn Buri,Nong Khaem,Thorn Buri,13.704722,100.348889,10160,"หลักสอง,หนองแขม,หนองค้างพลู",0,1,Convenience Store,Market,Thai Restaurant,Noodle House,Som Tum Restaurant


In [120]:
cluster2 = merged_group.loc[merged_group['Cluster Labels'] == 2, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster2.shape)
cluster2

(14, 14)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Thorn Buri,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง",0,2,Noodle House,Convenience Store,Coffee Shop,Japanese Restaurant,Asian Restaurant
6,Phra Nakhon,Bang Na,Phra Nakhon,13.680081,100.591800,10260,"บางนา,บางนาเหนือ,บางนาใต้",1,2,Café,Noodle House,Thai Restaurant,Coffee Shop,Asian Restaurant
9,Phra Nakhon,Bang Sue,Phra Nakhon,13.809722,100.537222,10800,"บางซื่อ,วงศ์สว่าง",1,2,Noodle House,Thai Restaurant,Convenience Store,Train Station,Ice Cream Shop
10,Thorn Buri,Bangkok Noi,Thorn Buri,13.770867,100.467933,10700,"บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บ...",0,2,Noodle House,Thai Restaurant,Asian Restaurant,Convenience Store,Market
11,Thorn Buri,Bangkok Yai,Thorn Buri,13.722778,100.476389,10600,"วัดอรุณ,วัดท่าพระ",0,2,Noodle House,Asian Restaurant,Convenience Store,Seafood Restaurant,Dessert Shop
15,Phra Nakhon,Din Daeng,Phra Nakhon,13.769722,100.552778,10400,"ดินแดง,รัชดาภิเษก",1,2,Noodle House,Thai Restaurant,Convenience Store,Park,Sports Club
17,Phra Nakhon,Dusit,Phra Nakhon,13.776944,100.520556,10300,"ดุสิต,วชิรพยาบาล,สวนจิตรลดา,สี่แยกมหานาค,บางซื...",1,2,Noodle House,Thai Restaurant,Convenience Store,Asian Restaurant,Coffee Shop
34,Phra Nakhon,Pom Prap Sattru Phai,Phra Nakhon,13.758056,100.513056,10100,"ป้อมปราบ,วัดเทพศิรินทร์,คลองมหานาค,บ้านบาตร,วั...",1,2,Noodle House,Thai Restaurant,Café,Asian Restaurant,Chinese Restaurant
35,Phra Nakhon,Prawet,Phra Nakhon,13.716944,100.694444,10250,"ประเวศ,หนองบอน,ดอกไม้,สวนหลวง",1,2,Convenience Store,Noodle House,Soccer Field,Electronics Store,Halal Restaurant
38,Phra Nakhon,Sai Mai,Phra Nakhon,13.919167,100.645833,10220,"สายไหม,ออเงิน,คลองถนน",1,2,Thai Restaurant,Noodle House,Convenience Store,Market,Café


In [121]:
cluster3 = merged_group.loc[merged_group['Cluster Labels'] == 3, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster3.shape)
cluster3

(1, 14)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Thorn Buri,Thawi Watthana,Thorn Buri,13.77263,100.353505,10170,"ทวีวัฒนา,ศาลาธรรมสพน์",0,3,Thai Restaurant,Soccer Field,Café,Restaurant,Vineyard


In [122]:
cluster4 = merged_group.loc[merged_group['Cluster Labels'] == 4, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster4.shape)
cluster4

(1, 14)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
46,Thorn Buri,Thung Khru,Thorn Buri,13.62542,100.493783,10140,"บางมด,ทุ่งครุ",0,4,Spa,Noodle House,Soccer Field,Convenience Store,Bakery


---

#### Let's look only at Phra Nakhon Side

In [64]:
bkk_phranakhon_side = bkk_location[bkk_location['Side'].str.contains('Phra Nakhon')]
bkk_phranakhon_side.drop(columns = 'Cluster', inplace = True)
bkk_phranakhon_side.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,District,Side,Latitude,Longitude,PostalCode,Sub-district
1,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง..."
3,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย..."
4,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่"
6,Bang Na,Phra Nakhon,13.680081,100.591800,10260,"บางนา,บางนาเหนือ,บางนาใต้"
8,Bang Rak,Phra Nakhon,13.730833,100.524167,10500,"มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา"


In [67]:
Phranakhonside_venues = getNearbyVenues(names=bkk_phranakhon_side['Sub-district'],
                                   latitudes=bkk_phranakhon_side['Latitude'],
                                   longitudes=bkk_phranakhon_side['Longitude']
                                  )

คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก
ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สายไหม,ทุ่งสองห้อง,ท่าแร้ง,ออเงิน
บางคอแหลม,วัดพระยาไกร,บางโคล่
บางนา,บางนาเหนือ,บางนาใต้
มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา
บางซื่อ,วงศ์สว่าง
คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์
ลาดยาว,เสนานิคม,จันทรเกษม,จอมพล,จตุจักร
ดินแดง,รัชดาภิเษก
ตลาดบางเขน,สีกัน,ทุ่งสองห้อง,ดอนเมือง,สนามบิน
ดุสิต,วชิรพยาบาล,สวนจิตรลดา,สี่แยกมหานาค,บางซื่อ,ถนนนครไชยศรี
ห้วยขวาง,บางกะปิ,ดินแดง,สามเสนนอก
คันนายาว,รามอินทรา
สามวาตะวันตก,สามวาตะวันออก,บางชัน,ทรายกองดิน,ทรายกองดินใต้
คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ
ทุ่งสองห้อง,ตลาดบางเขน
ลาดกระบัง,คลองสองต้นนุ่น,คลองสามประเวศ,ลำปลาทิว,ทับยาว,ขุมทอง
ลาดพร้าว,จรเข้บัว
มีนบุรี,แสนแสบ,บางชัน,ทรายกองดิน,ทรายกองดินใต้,สามวาตะวันออก,สามวาตะวันตก
กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง
รองเมือง,วังใหม่,ปทุมวัน,ลุมพินี
สามเสนใน,ถนนเพชรบุรี,ทุ่งพญาไท,มักกะสัน,ถนนพญาไท,พญาไท
คลองเ

In [68]:
print(Phranakhonside_venues.shape)
Phranakhonside_venues.head()

(2277, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,ตลาดนัดแฟลตคลองจั่น,13.770105,100.649124,Flea Market
1,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,ซ้งโภชนา ลูกชิ้นปลาทำเอง,13.761782,100.648010,Noodle House
2,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,Suan Son (สวนสน),13.762940,100.648895,Park
3,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,Burger King (เบอร์เกอร์ คิง),13.767083,100.643001,Fast Food Restaurant
4,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,ข้าวแกงปักษ์ใต้ แม่ยาใจ,13.770822,100.648628,Thai Restaurant


In [69]:
#There are 164 uniques venues categories from cols = ['Venue Category'] 
print('There are {} uniques venues categories.'.format(len(Phranakhonside_venues['Venue Category'].unique())))
#Group Bangkok_venues by cols =['Neighborhood'] and count all of the cols 
Phranakhonside_venues.groupby('Neighborhood').count().head()

There are 218 uniques venues categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง",19,19,19,19,19,19
"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",18,18,18,18,18,18
"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก",100,100,100,100,100,100
"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",23,23,23,23,23,23
"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,หนองบอน,ประเวศ,ดอกไม้,พระโขนงใต้",90,90,90,90,90,90


In [70]:
Phranakhonside_venues['Venue Category'].value_counts()

Thai Restaurant                  196
Noodle House                     184
Coffee Shop                      152
Café                             107
Convenience Store                104
Hotel                             78
Asian Restaurant                  72
Japanese Restaurant               67
Chinese Restaurant                60
Bar                               59
Dessert Shop                      41
Restaurant                        40
Som Tum Restaurant                36
Bakery                            34
Fast Food Restaurant              32
Steakhouse                        30
Hotpot Restaurant                 30
Seafood Restaurant                30
Shopping Mall                     29
BBQ Joint                         28
Ice Cream Shop                    27
Hostel                            27
Supermarket                       23
Gym / Fitness Center              21
Spa                               20
Pub                               19
Massage Studio                    19
M

In [71]:
# one hot encoding
Phranakhonside_endcode = pd.get_dummies(Phranakhonside_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood cols
Phranakhonside_endcode['Neighborhood'] = Phranakhonside_venues['Neighborhood']
Phranakhonside_endcode.head()

,Accessories Store,African Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,...,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
cols_list = Phranakhonside_endcode.columns.tolist()
# move Neighborhood to index 0 of cols_list
cols_list.insert(0, cols_list.pop(cols_list.index('Neighborhood')))
cols_list
# Reindex cols in bkk_endcode by assign to cols_list and assign it back to bkk_endcode
Phranakhonside_endcode = Phranakhonside_endcode.reindex(columns = cols_list)

Phranakhon_grouped = Phranakhonside_endcode.groupby('Neighborhood').mean().reset_index().round(2)
print(Phranakhon_grouped.shape)
Phranakhon_grouped.head()

(35, 218)


,Neighborhood,Accessories Store,African Restaurant,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Tunnel,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Zoo,Zoo Exhibit
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0
1,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.06,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0
2,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.00,...,0.0,0.0,0.00,0.01,0.01,0.01,0.0,0.01,0.0,0.0
3,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.00,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0
4,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.01,0.00,0.00,0.00,0.0,0.00,0.0,0.0


In [73]:
# Assign 5 to num_to_venues 
num_top_venues = 5
# Create function that create temp that contain venue and freq of each hood in neighborhood 
for hood in Phranakhon_grouped['Neighborhood']:
    print("----"+hood+"----") #
    temp = Phranakhon_grouped[Phranakhon_grouped['Neighborhood'] == hood].T.reset_index() #reset index
    temp.columns = ['venue','freq'] # create 2 temp col (venue, freq)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง----
             venue  freq
0      Coffee Shop  0.16
1    Shopping Mall  0.11
2       Food Court  0.05
3        Bookstore  0.05
4  Thai Restaurant  0.05


----คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์----
               venue  freq
0        Coffee Shop  0.17
1               Café  0.11
2        Supermarket  0.11
3  Convenience Store  0.11
4        Flea Market  0.06


----คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก----
                  venue  freq
0           Coffee Shop  0.08
1       Thai Restaurant  0.08
2   Japanese Restaurant  0.07
3  Fast Food Restaurant  0.05
4          Dessert Shop  0.04


----คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ----
               venue  freq
0        Coffee Shop  0.17
1  Convenience Store  0.17
2        Karaoke Bar  0.04
3                Bar  0.04
4              Canal  0.04


----คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลว

In [74]:
# Create a function that return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [75]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Phranakhon_grouped['Neighborhood']

for ind in np.arange(Phranakhon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Phranakhon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",Coffee Shop,Shopping Mall,Doctor's Office,Convenience Store,Snack Place
1,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",Coffee Shop,Supermarket,Convenience Store,Café,Gym / Fitness Center
2,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Dessert Shop
3,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",Convenience Store,Coffee Shop,Residential Building (Apartment / Condo),Noodle House,Thai Restaurant
4,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",Convenience Store,Coffee Shop,Noodle House,Hotel,Fast Food Restaurant


In [76]:
# k = 5 
k = 5
# Drop col Neighborhood and assign to cluster
cluster = Phranakhon_grouped.drop('Neighborhood', axis = 1)
# k-means Clustering
clustering = KMeans(n_clusters = k, random_state = 1).fit(cluster)
#check cluster label
clustering.labels_[0:10]

array([4, 4, 4, 3, 3, 1, 4, 4, 0, 0], dtype=int32)

In [77]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', clustering.labels_)
merged_group = bkk_phranakhon_side
#join
merged_group = merged_group.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on= 'Sub-district')
merged_group.head()

,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",4,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Dessert Shop
3,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย...",3,Fast Food Restaurant,Coffee Shop,Convenience Store,Noodle House,Asian Restaurant
4,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่",3,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant
6,Bang Na,Phra Nakhon,13.680081,100.591800,10260,"บางนา,บางนาเหนือ,บางนาใต้",1,Thai Restaurant,Noodle House,Café,Pier,Asian Restaurant
8,Bang Rak,Phra Nakhon,13.730833,100.524167,10500,"มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา",1,Noodle House,Hotel,Thai Restaurant,Chinese Restaurant,Café


In [78]:
merged_group['Cluster Labels'].value_counts()

4    14
1     7
0     7
3     6
2     1
Name: Cluster Labels, dtype: int64

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for a, b, c, d, e, f in zip(merged_group['Latitude'], merged_group['Longitude'], merged_group['PostalCode'], merged_group['District'], merged_group['Sub-district'], merged_group['Cluster Labels']):
    label = folium.Popup(str(d) + ' Cluster ' + str(f), parse_html=True)
    folium.CircleMarker(
        [a, b],
        radius=5,
        popup=label,
        color=rainbow[f-1],
        fill=True,
        fill_color=rainbow[f-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [80]:
cluster0 = merged_group.loc[merged_group['Cluster Labels'] == 0, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster0.shape)
cluster0

(7, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Phra Nakhon,Bang Sue,Phra Nakhon,13.809722,100.537222,10800,"บางซื่อ,วงศ์สว่าง",0,Noodle House,Thai Restaurant,Convenience Store,Train Station,Ice Cream Shop
15,Phra Nakhon,Din Daeng,Phra Nakhon,13.769722,100.552778,10400,"ดินแดง,รัชดาภิเษก",0,Noodle House,Thai Restaurant,Convenience Store,Sports Club,Park
17,Phra Nakhon,Dusit,Phra Nakhon,13.776944,100.520556,10300,"ดุสิต,วชิรพยาบาล,สวนจิตรลดา,สี่แยกมหานาค,บางซื...",0,Noodle House,Thai Restaurant,Convenience Store,Coffee Shop,Asian Restaurant
35,Phra Nakhon,Prawet,Phra Nakhon,13.716944,100.694444,10250,"ประเวศ,หนองบอน,ดอกไม้,สวนหลวง",0,Convenience Store,Noodle House,Soccer Field,Food Stand,Thai Restaurant
38,Phra Nakhon,Sai Mai,Phra Nakhon,13.919167,100.645833,10220,"สายไหม,ออเงิน,คลองถนน",0,Thai Restaurant,Convenience Store,Noodle House,Café,Market
41,Phra Nakhon,Sathon,Phra Nakhon,13.708056,100.526389,10120,"ทุ่งวัดดอน,ยานนาวา,ทุ่งมหาเมฆ",0,Noodle House,Asian Restaurant,Chinese Restaurant,Convenience Store,Café
42,Phra Nakhon,Suan Luang,Phra Nakhon,13.730278,100.651389,10250,"สวนหลวง,อ่อนนุช,พัฒนาการ",0,Noodle House,Coffee Shop,Thai Restaurant,Asian Restaurant,Convenience Store


In [81]:
cluster1 = merged_group.loc[merged_group['Cluster Labels'] == 1, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster1.shape)
cluster1

(7, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Phra Nakhon,Bang Na,Phra Nakhon,13.680081,100.591800,10260,"บางนา,บางนาเหนือ,บางนาใต้",1,Thai Restaurant,Noodle House,Café,Pier,Asian Restaurant
8,Phra Nakhon,Bang Rak,Phra Nakhon,13.730833,100.524167,10500,"มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา",1,Noodle House,Hotel,Thai Restaurant,Chinese Restaurant,Café
24,Phra Nakhon,Lat Krabang,Phra Nakhon,13.722317,100.759669,10520,"ลาดกระบัง,คลองสองต้นนุ่น,คลองสามประเวศ,ลำปลาทิ...",1,Thai Restaurant,Café,Hotel,Som Tum Restaurant,Bar
33,Phra Nakhon,Phra Nakhon,Phra Nakhon,13.764444,100.499167,10200,"พระบรมมหาราชวัง,วังบูรพาภิรมย์,วัดราชบพิธ,สำรา...",1,Bar,Thai Restaurant,Hotel,Café,Hostel
34,Phra Nakhon,Pom Prap Sattru Phai,Phra Nakhon,13.758056,100.513056,10100,"ป้อมปราบ,วัดเทพศิรินทร์,คลองมหานาค,บ้านบาตร,วั...",1,Noodle House,Thai Restaurant,Café,Asian Restaurant,Chinese Restaurant
47,Phra Nakhon,Wang Thonglang,Phra Nakhon,13.754253,100.493087,10310,"วังทองหลาง,สะพานสอง,คลองเจ้าคุณสิงห์,พลับพลา",1,Café,Bar,Thai Restaurant,Palace,Noodle House
48,Phra Nakhon,Watthana,Phra Nakhon,13.742222,100.585833,10110,"คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",1,Café,Coffee Shop,Thai Restaurant,Bar,Noodle House


In [82]:
cluster2 = merged_group.loc[merged_group['Cluster Labels'] == 2, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster2.shape)
cluster2

(1, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,Phra Nakhon,Min Buri,Phra Nakhon,13.813889,100.748056,10510,"มีนบุรี,แสนแสบ,บางชัน,ทรายกองดิน,ทรายกองดินใต้...",2,Intersection,Coffee Shop,Restaurant,Bakery,Convenience Store


---

In [83]:
cluster3 = merged_group.loc[merged_group['Cluster Labels'] == 3, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster3.shape)
cluster3

(6, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Phra Nakhon,Bang Khen,Phra Nakhon,13.873889,100.596389,10220,"ลาดยาว,อนุสาวรีย์,คลองถนน,ตลาดบางเขน,สีกัน,สาย...",3,Fast Food Restaurant,Coffee Shop,Convenience Store,Noodle House,Asian Restaurant
4,Phra Nakhon,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,"บางคอแหลม,วัดพระยาไกร,บางโคล่",3,Convenience Store,Noodle House,Coffee Shop,Thai Restaurant,Chinese Restaurant
16,Phra Nakhon,Don Mueang,Phra Nakhon,13.913611,100.589722,10210,"ตลาดบางเขน,สีกัน,ทุ่งสองห้อง,ดอนเมือง,สนามบิน",3,Convenience Store,Food Truck,Noodle House,Grocery Store,Thai Restaurant
22,Phra Nakhon,Khlong Toei,Phra Nakhon,13.708056,100.583889,10110,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",3,Convenience Store,Coffee Shop,Residential Building (Apartment / Condo),Noodle House,Thai Restaurant
32,Phra Nakhon,Phra Khanong,Phra Nakhon,13.702222,100.601667,10260,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",3,Convenience Store,Coffee Shop,Noodle House,Hotel,Fast Food Restaurant
37,Phra Nakhon,Ratchathewi,Phra Nakhon,13.758889,100.534444,10400,"ทุ่งพญาไท,ถนนพญาไท,ถนนเพชรบุรี,มักกะสัน",3,Hotel,Noodle House,Coffee Shop,Steakhouse,Hostel


In [84]:
cluster4 = merged_group.loc[merged_group['Cluster Labels'] == 4, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster4.shape)
cluster4

(14, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Phra Nakhon,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",4,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Dessert Shop
12,Phra Nakhon,Bueng Kum,Phra Nakhon,13.785278,100.669167,10240,"คลองกุ่ม,สะพานสูง,คันนายาว,นวมินทร์,นวลจันทร์",4,Coffee Shop,Supermarket,Convenience Store,Café,Gym / Fitness Center
13,Phra Nakhon,Chatuchak,Phra Nakhon,13.828611,100.559722,10900,"ลาดยาว,เสนานิคม,จันทรเกษม,จอมพล,จตุจักร",4,Thai Restaurant,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Noodle House
18,Phra Nakhon,Huai Khwang,Phra Nakhon,13.776667,100.579444,10310,"ห้วยขวาง,บางกะปิ,ดินแดง,สามเสนนอก",4,Thai Restaurant,Noodle House,Japanese Restaurant,Hotel,Convenience Store
19,Phra Nakhon,Khan Na Yao,Phra Nakhon,13.818881,100.678848,10230,"คันนายาว,รามอินทรา",4,Thai Restaurant,Coffee Shop,Japanese Restaurant,Som Tum Restaurant,Dessert Shop
20,Phra Nakhon,Khlong Sam Wa,Phra Nakhon,13.859722,100.704167,10510,"สามวาตะวันตก,สามวาตะวันออก,บางชัน,ทรายกองดิน,ท...",4,Restaurant,Thai Restaurant,Exhibit,Noodle House,Coffee Shop
23,Phra Nakhon,Lak Si,Phra Nakhon,13.887500,100.578889,10210,"ทุ่งสองห้อง,ตลาดบางเขน",4,Convenience Store,Coffee Shop,Thai Restaurant,Japanese Restaurant,Food Truck
25,Phra Nakhon,Lat Phrao,Phra Nakhon,13.803611,100.607500,10230,"ลาดพร้าว,จรเข้บัว",4,Noodle House,Thai Restaurant,Café,Som Tum Restaurant,Coffee Shop
27,Phra Nakhon,Nong Chok,Phra Nakhon,13.855556,100.862500,10530,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",4,Coffee Shop,Shopping Mall,Doctor's Office,Convenience Store,Snack Place
29,Phra Nakhon,Pathum Wan,Phra Nakhon,13.744942,100.522200,10330,"รองเมือง,วังใหม่,ปทุมวัน,ลุมพินี",4,Noodle House,Asian Restaurant,Hostel,Chinese Restaurant,Thai Restaurant


#### Let's look only thorn buri side

In [86]:
bkk_thornburi_side = bkk_location[bkk_location['Side'].str.contains('Thorn Buri')]
bkk_thornburi_side.drop(columns = 'Cluster', inplace = True)
bkk_thornburi_side.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,District,Side,Latitude,Longitude,PostalCode,Sub-district
0,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
2,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง"
5,Bang Khun Thian,Thorn Buri,13.660833,100.435833,10150,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด..."
7,Bang Phlat,Thorn Buri,13.793889,100.505000,10700,"บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน"
10,Bangkok Noi,Thorn Buri,13.770867,100.467933,10700,"บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บ..."


In [87]:
Thornburiside_venues = getNearbyVenues(names=bkk_thornburi_side['Sub-district'],
                                   latitudes=bkk_thornburi_side['Latitude'],
                                   longitudes=bkk_thornburi_side['Longitude']
                                  )

บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบางบอน
บางแค,บางแคเหนือ,บางไผ่,หลักสอง
บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด,แสมดำ
บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน
บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บางขุนนนท์,บางขุนศรี,บางยี่ขัน,อรุณอมรินทร์
วัดอรุณ,วัดท่าพระ
บางขุนเทียน,บางค้อ,บางมด,จอมทอง
สมเด็จเจ้าพระยา,คลองสาน,บางลำภูล่าง,คลองต้นไทร
หลักสอง,หนองแขม,หนองค้างพลู
บางหว้า,บางด้วน,บางแค,บางแคเหนือ,บางไผ่,บางจาก,บางแวก,คลองขวาง,ปากคลองภาษีเจริญ,คูหาสวรรค์
ราษฎร์บูรณะ,บางปะกอก,บางมด,ทุ่งครุ
คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวีวัฒนา,บางเชือกหนัง,ศาลาธรรมสพน์
ทวีวัฒนา,ศาลาธรรมสพน์
วัดกัลยาณ์,หิรัญรูจี,บางยี่เรือ,บุคคโล,ตลาดพลู,ดาวคะนอง,สำเหร่
บางมด,ทุ่งครุ


In [88]:
print(Thornburiside_venues.shape)
Thornburiside_venues.head()

(605, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,Shabu Na-Mor (ชาบูหน้าหม้อ),13.663855,100.428756,Shabu-Shabu Restaurant
1,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ก๋วยเตี๋ยวตำลึง นายเอก,13.667102,100.428354,Noodle House
2,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ส้มตำครกเงิน,13.666023,100.432755,Som Tum Restaurant
3,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ไผ่ทองโภชนา,13.662101,100.435264,Thai Restaurant
4,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ขนมจีนเทวดา บีบเส้นสดๆ,13.659428,100.433692,Noodle House


In [89]:
Thornburiside_venues['Venue Category'].value_counts()

Noodle House                 60
Thai Restaurant              50
Convenience Store            37
Coffee Shop                  31
Asian Restaurant             26
Chinese Restaurant           19
Seafood Restaurant           18
Japanese Restaurant          16
Café                         15
Restaurant                   14
Dessert Shop                 12
Fast Food Restaurant         11
Steakhouse                   10
Bakery                       10
Soccer Field                  9
Ice Cream Shop                9
Hotpot Restaurant             9
Shopping Mall                 9
Pizza Place                   8
BBQ Joint                     8
Department Store              8
Spa                           8
Bar                           8
Supermarket                   7
Shabu-Shabu Restaurant        7
Market                        6
Train Station                 5
Som Tum Restaurant            5
Hotel                         5
Badminton Court               5
                             ..
Shop & S

In [90]:
# one hot encoding
Thornburiside_endcode = pd.get_dummies(Thornburiside_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood cols
Thornburiside_endcode['Neighborhood'] = Thornburiside_venues['Neighborhood']
Thornburiside_endcode.head()

,Art Gallery,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,...,Tea Room,Thai Restaurant,Theme Park,Toll Plaza,Tour Provider,Train Station,Video Store,Vietnamese Restaurant,Vineyard,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ..."


In [91]:
cols_list = Thornburiside_endcode.columns.tolist()
# move Neighborhood to index 0 of cols_list
cols_list.insert(0, cols_list.pop(cols_list.index('Neighborhood')))
cols_list
# Reindex cols in bkk_endcode by assign to cols_list and assign it back to bkk_endcode
Thornburiside_endcode = Thornburiside_endcode.reindex(columns = cols_list)

Thornburi_grouped = Thornburiside_endcode.groupby('Neighborhood').mean().reset_index().round(2)
print(Thornburi_grouped.shape)
Thornburi_grouped.head()

(15, 119)


,Neighborhood,Art Gallery,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,...,Taxi Stand,Tea Room,Thai Restaurant,Theme Park,Toll Plaza,Tour Provider,Train Station,Video Store,Vietnamese Restaurant,Vineyard
0,"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวี...",0.0,0.02,0.00,0.02,0.02,0.00,0.00,0.0,0.02,...,0.0,0.0,0.07,0.0,0.02,0.0,0.0,0.0,0.0,0.00
1,"ทวีวัฒนา,ศาลาธรรมสพน์",0.0,0.00,0.00,0.00,0.07,0.00,0.07,0.0,0.00,...,0.0,0.0,0.33,0.0,0.00,0.0,0.0,0.0,0.0,0.07
2,"บางขุนเทียน,บางค้อ,บางมด,จอมทอง",0.0,0.06,0.00,0.00,0.00,0.06,0.00,0.0,0.00,...,0.0,0.0,0.19,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",0.0,0.04,0.01,0.00,0.04,0.02,0.00,0.0,0.00,...,0.0,0.0,0.08,0.0,0.00,0.0,0.0,0.0,0.0,0.00
4,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",0.0,0.04,0.00,0.04,0.07,0.00,0.00,0.0,0.00,...,0.0,0.0,0.09,0.0,0.00,0.0,0.0,0.0,0.0,0.00


In [92]:
# Assign 5 to num_to_venues 
num_top_venues = 5
# Create function that create temp that contain venue and freq of each hood in neighborhood 
for hood in Thornburi_grouped['Neighborhood']:
    print("----"+hood+"----") #
    temp = Thornburi_grouped[Thornburi_grouped['Neighborhood'] == hood].T.reset_index() #reset index
    temp.columns = ['venue','freq'] # create 2 temp col (venue, freq)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวีวัฒนา,บางเชือกหนัง,ศาลาธรรมสพน์----
               venue  freq
0  Convenience Store  0.14
1       Noodle House  0.14
2               Café  0.07
3    Thai Restaurant  0.07
4        Coffee Shop  0.07


----ทวีวัฒนา,ศาลาธรรมสพน์----
             venue  freq
0  Thai Restaurant  0.33
1     Soccer Field  0.13
2             Café  0.13
3         Vineyard  0.07
4   Baseball Field  0.07


----บางขุนเทียน,บางค้อ,บางมด,จอมทอง----
               venue  freq
0    Thai Restaurant  0.19
1  Convenience Store  0.19
2        Flea Market  0.06
3          Pet Store  0.06
4        Coffee Shop  0.06


----บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด,แสมดำ----
                 venue  freq
0      Thai Restaurant  0.08
1          Coffee Shop  0.06
2           Restaurant  0.06
3  Japanese Restaurant  0.05
4       Ice Cream Shop  0.04


----บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบางบอน----
                    venue  freq
0         Thai Restaurant  0.09
1  

In [93]:
# Create a function that return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Thornburi_grouped['Neighborhood']

for ind in np.arange(Thornburi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Thornburi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวี...",Convenience Store,Noodle House,Café,Coffee Shop,Thai Restaurant
1,"ทวีวัฒนา,ศาลาธรรมสพน์",Thai Restaurant,Soccer Field,Café,Restaurant,Gym
2,"บางขุนเทียน,บางค้อ,บางมด,จอมทอง",Convenience Store,Thai Restaurant,Fried Chicken Joint,Food Truck,Chinese Restaurant
3,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",Thai Restaurant,Coffee Shop,Restaurant,Japanese Restaurant,Ice Cream Shop
4,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",Thai Restaurant,Noodle House,Bakery,Pizza Place,Coffee Shop


In [95]:
# k = 5 
k = 5
# Drop col Neighborhood and assign to cluster
cluster = Thornburi_grouped.drop('Neighborhood', axis = 1)
# k-means Clustering
clustering = KMeans(n_clusters = k, random_state = 1).fit(cluster)
#check cluster label
clustering.labels_[0:10]

array([0, 3, 4, 1, 1, 0, 1, 2, 1, 0], dtype=int32)

In [96]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', clustering.labels_)
merged_group = bkk_thornburi_side
#join
merged_group = merged_group.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on= 'Sub-district')
merged_group.head()

,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",1,Thai Restaurant,Noodle House,Bakery,Pizza Place,Coffee Shop
2,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง",0,Noodle House,Convenience Store,Shopping Mall,Asian Restaurant,Coffee Shop
5,Bang Khun Thian,Thorn Buri,13.660833,100.435833,10150,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",1,Thai Restaurant,Coffee Shop,Restaurant,Japanese Restaurant,Ice Cream Shop
7,Bang Phlat,Thorn Buri,13.793889,100.505000,10700,"บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน",1,Thai Restaurant,Convenience Store,Fast Food Restaurant,Café,Department Store
10,Bangkok Noi,Thorn Buri,13.770867,100.467933,10700,"บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บ...",0,Noodle House,Asian Restaurant,Thai Restaurant,Convenience Store,Road


In [97]:
merged_group['Cluster Labels'].value_counts()

1    6
0    5
4    2
3    1
2    1
Name: Cluster Labels, dtype: int64

In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for a, b, c, d, e, f in zip(merged_group['Latitude'], merged_group['Longitude'], merged_group['PostalCode'], merged_group['District'], merged_group['Sub-district'], merged_group['Cluster Labels']):
    label = folium.Popup(str(d) + ' Cluster ' + str(f), parse_html=True)
    folium.CircleMarker(
        [a, b],
        radius=5,
        popup=label,
        color=rainbow[f-1],
        fill=True,
        fill_color=rainbow[f-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [99]:
cluster0 = merged_group.loc[merged_group['Cluster Labels'] == 0, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster0.shape)
cluster0

(5, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Thorn Buri,Bang Khae,Thorn Buri,13.696111,100.409444,10160,"บางแค,บางแคเหนือ,บางไผ่,หลักสอง",0,Noodle House,Convenience Store,Shopping Mall,Asian Restaurant,Coffee Shop
10,Thorn Buri,Bangkok Noi,Thorn Buri,13.770867,100.467933,10700,"บางพลัด,บางบำหรุ,บางอ้อ,ศิริราช,บ้านช่างหล่อ,บ...",0,Noodle House,Asian Restaurant,Thai Restaurant,Convenience Store,Road
11,Thorn Buri,Bangkok Yai,Thorn Buri,13.722778,100.476389,10600,"วัดอรุณ,วัดท่าพระ",0,Noodle House,Asian Restaurant,Convenience Store,Farmers Market,Coffee Shop
43,Thorn Buri,Taling Chan,Thorn Buri,13.776944,100.456667,10170,"คลองชักพระ,ตลิ่งชัน,ฉิมพลี,บางพรม,บางระมาด,ทวี...",0,Convenience Store,Noodle House,Café,Coffee Shop,Thai Restaurant
45,Thorn Buri,Thon Buri,Thorn Buri,13.725000,100.485833,10600,"วัดกัลยาณ์,หิรัญรูจี,บางยี่เรือ,บุคคโล,ตลาดพลู...",0,Noodle House,Convenience Store,Asian Restaurant,Thai Restaurant,Train Station


In [100]:
cluster1 = merged_group.loc[merged_group['Cluster Labels'] == 1, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster1.shape)
cluster1

(6, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Thorn Buri,Bang Bon,Thorn Buri,13.666503,100.428859,10150,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",1,Thai Restaurant,Noodle House,Bakery,Pizza Place,Coffee Shop
5,Thorn Buri,Bang Khun Thian,Thorn Buri,13.660833,100.435833,10150,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",1,Thai Restaurant,Coffee Shop,Restaurant,Japanese Restaurant,Ice Cream Shop
7,Thorn Buri,Bang Phlat,Thorn Buri,13.793889,100.505000,10700,"บางพลัด,บางอ้อ,บางบำหรุ,บางยี่ขัน",1,Thai Restaurant,Convenience Store,Fast Food Restaurant,Café,Department Store
21,Thorn Buri,Khlong San,Thorn Buri,13.730278,100.509722,10600,"สมเด็จเจ้าพระยา,คลองสาน,บางลำภูล่าง,คลองต้นไทร",1,Chinese Restaurant,Noodle House,Dessert Shop,Thai Restaurant,Coffee Shop
30,Thorn Buri,Phasi Charoen,Thorn Buri,13.714722,100.437222,10160,"บางหว้า,บางด้วน,บางแค,บางแคเหนือ,บางไผ่,บางจาก...",1,Coffee Shop,Convenience Store,Japanese Restaurant,Thai Restaurant,BBQ Joint
36,Thorn Buri,Rat Burana,Thorn Buri,13.682222,100.505556,10140,"ราษฎร์บูรณะ,บางปะกอก,บางมด,ทุ่งครุ",1,Thai Restaurant,Noodle House,Coffee Shop,Chinese Restaurant,Hotpot Restaurant


In [101]:
cluster2 = merged_group.loc[merged_group['Cluster Labels'] == 2, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster2.shape)
cluster2

(1, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
46,Thorn Buri,Thung Khru,Thorn Buri,13.62542,100.493783,10140,"บางมด,ทุ่งครุ",2,Spa,Bakery,Convenience Store,Soccer Field,Noodle House


In [102]:
cluster3 = merged_group.loc[merged_group['Cluster Labels'] == 3, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster3.shape)
cluster3

(1, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Thorn Buri,Thawi Watthana,Thorn Buri,13.77263,100.353505,10170,"ทวีวัฒนา,ศาลาธรรมสพน์",3,Thai Restaurant,Soccer Field,Café,Restaurant,Gym


In [103]:
cluster4 = merged_group.loc[merged_group['Cluster Labels'] == 4, merged_group.columns[[1] + list(range(0, merged_group.shape[1]))]]
print(cluster4.shape)
cluster4

(2, 13)


,Side,District,Side,Latitude,Longitude,PostalCode,Sub-district,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Thorn Buri,Chom Thong,Thorn Buri,13.677222,100.484722,10150,"บางขุนเทียน,บางค้อ,บางมด,จอมทอง",4,Convenience Store,Thai Restaurant,Fried Chicken Joint,Food Truck,Chinese Restaurant
28,Thorn Buri,Nong Khaem,Thorn Buri,13.704722,100.348889,10160,"หลักสอง,หนองแขม,หนองค้างพลู",4,Convenience Store,Market,Som Tum Restaurant,Noodle House,Dessert Shop


---

#### Now only let's look at Venue Category = Night club

In [123]:
def getNearbyVenues2(names, latitudes, longitudes, radius=1000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT, categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass
                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues2 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues2.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues2)

    return(nearby_venues2)

In [140]:
bkk_bar_venues = getNearbyVenues2(names = bkk_location['Sub-district'],
                                   latitudes= bkk_location['Latitude'],
                                   longitudes = bkk_location['Longitude'],
                                 categoryIds = '4bf58dd8d48988d11f941735')
print(bkk_bar_venues.shape)
bkk_bar_venues.head()

(296, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,RM2 Club,13.665800,100.428997,Nightclub
1,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,vintage,13.660642,100.429293,Nightclub
2,"บางบอน,บางบอนเหนือ,บางบอนใต้,คลองบางพราน,คลองบ...",13.666503,100.428859,ศรีรัตน์พัฒนา,13.675447,100.428840,Nightclub
3,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,ICK Pub,13.762867,100.644091,Nightclub
4,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",13.765833,100.647778,G-STAR Paradise Pub,13.769842,100.645406,Nightclub


In [141]:
# one hot encoding
bkk_bar_endcode = pd.get_dummies(bkk_bar_venues[['Venue Category']], prefix="", prefix_sep="")
#add neighborhood cols
bkk_bar_endcode['Neighborhood'] = bkk_bar_venues['Neighborhood']
bkk_bar_grouped  = bkk_bar_endcode.groupby('Neighborhood').mean().reset_index().round(2)
bkk_bar_grouped.head()

,Neighborhood,Bar,Brewery,Cocktail Bar,Gay Bar,Health & Beauty Service,Lounge,Music Venue,Nightclub,Other Nightlife,Pub,Speakeasy,Strip Club,Whisky Bar
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",0.00,0.0,0.0,0.0,0.0,0.00,0.0,1.00,0.0,0.0,0.0,0.0,0.00
1,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",0.00,0.0,0.0,0.0,0.0,0.00,0.0,1.00,0.0,0.0,0.0,0.0,0.00
2,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.75,0.0,0.0,0.0,0.0,0.00
3,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",0.00,0.0,0.0,0.0,0.0,0.00,0.0,1.00,0.0,0.0,0.0,0.0,0.00
4,"คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",0.02,0.0,0.0,0.0,0.0,0.00,0.0,0.85,0.0,0.1,0.0,0.0,0.02


In [142]:
num_top_venues = 5
# Create function that create temp that contain venue and freq of each hood in neighborhood 
for hood in bkk_bar_grouped['Neighborhood']:
    print("----"+hood+"----") #print name of the neighborhood
    temp = bkk_bar_grouped[bkk_bar_grouped['Neighborhood'] == hood].T.reset_index() #reset index
    temp.columns = ['venue','freq'] # create 2 temp col (venue, freq)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,คู้ฝั่งเหนือ,ลำผักชี,ลำต้อยติ่ง----
          venue  freq
0     Nightclub   1.0
1           Bar   0.0
2       Brewery   0.0
3  Cocktail Bar   0.0
4       Gay Bar   0.0


----คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง,คันนายาว,จรเข้บัว,หัวหมาก----
          venue  freq
0     Nightclub   1.0
1           Bar   0.0
2       Brewery   0.0
3  Cocktail Bar   0.0
4       Gay Bar   0.0


----คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ----
          venue  freq
0     Nightclub  0.75
1        Lounge  0.25
2           Bar  0.00
3       Brewery  0.00
4  Cocktail Bar  0.00


----คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,หนองบอน,ประเวศ,ดอกไม้,พระโขนงใต้----
          venue  freq
0     Nightclub   1.0
1           Bar   0.0
2       Brewery   0.0
3  Cocktail Bar   0.0
4       Gay Bar   0.0


----คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ----
        venue  freq
0   Nightclub  0.85
1         Pub  0.10
2         Bar  0.02
3  Whisky Bar 

In [143]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_bar_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_bar_venues_sorted['Neighborhood'] = bkk_bar_grouped['Neighborhood']

for ind in np.arange(bkk_bar_grouped.shape[0]):
    neighborhoods_bar_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bkk_bar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_bar_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub
1,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub
2,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",Nightclub,Lounge,Whisky Bar,Strip Club,Speakeasy
3,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub
4,"คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",Nightclub,Pub,Whisky Bar,Bar,Strip Club


In [144]:
# k = 5
k = 5
# Drop col Neighborhood and assign to cluster
cluster = bkk_bar_grouped.drop('Neighborhood', axis = 1)
# k-means Clustering
kmeans = KMeans(n_clusters = k, random_state = 1).fit(cluster)
#check cluster label
print(kmeans.labels_[0:10])

[0 0 4 0 4 0 0 1 0 4]


In [145]:
neighborhoods_bar_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged = neighborhoods_bar_venues_sorted
merged = merged.join(bkk_location.set_index('Sub-district'), on= 'Neighborhood')
merged.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
0,0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Nong Chok,Phra Nakhon,13.855556,100.862500,10530,1
1,0,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,1
2,4,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",Nightclub,Lounge,Whisky Bar,Strip Club,Speakeasy,Khlong Toei,Phra Nakhon,13.708056,100.583889,10110,1
3,0,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Phra Khanong,Phra Nakhon,13.702222,100.601667,10260,1
4,4,"คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",Nightclub,Pub,Whisky Bar,Bar,Strip Club,Watthana,Phra Nakhon,13.742222,100.585833,10110,1


In [146]:
merged['Cluster Labels'].value_counts()

0    25
4     8
2     3
1     3
3     1
Name: Cluster Labels, dtype: int64

In [147]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for a, b, c, d, e, f in zip(merged['Latitude'], merged['Longitude'], merged['PostalCode'], merged['District'], merged['Neighborhood'], merged['Cluster Labels']):
    label = '{} {}'.format(d, e)
    label = folium.Popup(str(e) + ' Cluster ' + str(f), parse_html=True)
    folium.CircleMarker(
        [a, b],
        radius=5,
        popup=label,
        color=rainbow[f - 1],
        fill=True,
        fill_color=rainbow[f - 1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [148]:
cluster0 = merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(2, merged.shape[1]))]]
print(cluster0.shape)
cluster0

(25, 12)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
0,"กระทุ่มราย,หนองจอก,คลองสิบ,คลองสิบสอง,โคกแฝด,ค...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Nong Chok,Phra Nakhon,13.855556,100.862500,10530,1
1,"คลองจั่น,วังทองหลาง,ลาดพร้าว,คลองกุ่ม,สะพานสูง...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Bang Kapi,Phra Nakhon,13.765833,100.647778,10240,1
3,"คลองเตย,คลองตัน,พระโขนง,บางนา,บางจาก,สวนหลวง,ห...",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Phra Khanong,Phra Nakhon,13.702222,100.601667,10260,1
5,"จักรวรรดิ,สัมพันธวงศ์,ตลาดน้อย",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Samphanthawong,Phra Nakhon,13.731389,100.514167,10100,1
6,"ดินแดง,รัชดาภิเษก",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Din Daeng,Phra Nakhon,13.769722,100.552778,10400,1
8,"ตลาดบางเขน,สีกัน,ทุ่งสองห้อง,ดอนเมือง,สนามบิน",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Don Mueang,Phra Nakhon,13.913611,100.589722,10210,1
11,"ทุ่งวัดดอน,ยานนาวา,ทุ่งมหาเมฆ",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Sathon,Phra Nakhon,13.708056,100.526389,10120,1
12,"ทุ่งสองห้อง,ตลาดบางเขน",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Lak Si,Phra Nakhon,13.887500,100.578889,10210,1
13,"บางคอแหลม,วัดพระยาไกร,บางโคล่",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Bang Kho Laem,Phra Nakhon,13.693333,100.502500,10120,1
15,"บางซื่อ,วงศ์สว่าง",Nightclub,Whisky Bar,Strip Club,Speakeasy,Pub,Bang Sue,Phra Nakhon,13.809722,100.537222,10800,1


In [149]:
cluster1 = merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(2, merged.shape[1]))]]
print(cluster1.shape)
cluster1

(3, 12)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
7,"ดุสิต,วชิรพยาบาล,สวนจิตรลดา,สี่แยกมหานาค,บางซื...",Nightclub,Bar,Whisky Bar,Strip Club,Speakeasy,Dusit,Phra Nakhon,13.776944,100.520556,10300,1
10,"ทุ่งวัดดอน,ยานนาวา,ช่องนนทรี,บางโพงพาง,วัดพระย...",Nightclub,Bar,Whisky Bar,Strip Club,Speakeasy,Yan Nawa,Phra Nakhon,13.696944,100.543056,10120,1
27,"ลาดพร้าว,จรเข้บัว",Nightclub,Bar,Whisky Bar,Strip Club,Speakeasy,Lat Phrao,Phra Nakhon,13.803611,100.607500,10230,1


In [150]:
cluster2 = merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(2, merged.shape[1]))]]
print(cluster2.shape)
cluster2

(3, 12)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
22,"พระบรมมหาราชวัง,วังบูรพาภิรมย์,วัดราชบพิธ,สำรา...",Nightclub,Pub,Bar,Music Venue,Whisky Bar,Phra Nakhon,Phra Nakhon,13.764444,100.499167,10200,1
29,"วังทองหลาง,สะพานสอง,คลองเจ้าคุณสิงห์,พลับพลา",Nightclub,Pub,Bar,Music Venue,Whisky Bar,Wang Thonglang,Phra Nakhon,13.754253,100.493087,10310,1
31,"วัดอรุณ,วัดท่าพระ",Nightclub,Pub,Other Nightlife,Bar,Whisky Bar,Bangkok Yai,Thorn Buri,13.722778,100.476389,10600,0


In [151]:
cluster3 = merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(2, merged.shape[1]))]]
print(cluster3.shape)
cluster3

(1, 12)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
14,"บางค้อ,จอมทอง,บางขุนเทียน,บางบอน,ท่าข้าม,บางมด...",Nightclub,Brewery,Whisky Bar,Strip Club,Speakeasy,Bang Khun Thian,Thorn Buri,13.660833,100.435833,10150,0


In [152]:
cluster4 = merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(2, merged.shape[1]))]]
print(cluster4.shape)
cluster4

(8, 12)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,District,Side,Latitude,Longitude,PostalCode,Cluster
2,"คลองเตย,คลองตัน,พระโขนง,คลองเตยเหนือ,คลองตันเห...",Nightclub,Lounge,Whisky Bar,Strip Club,Speakeasy,Khlong Toei,Phra Nakhon,13.708056,100.583889,10110,1
4,"คลองเตยเหนือ,คลองตันเหนือ,พระโขนงเหนือ",Nightclub,Pub,Whisky Bar,Bar,Strip Club,Watthana,Phra Nakhon,13.742222,100.585833,10110,1
9,"ทุ่งพญาไท,ถนนพญาไท,ถนนเพชรบุรี,มักกะสัน",Nightclub,Pub,Whisky Bar,Strip Club,Speakeasy,Ratchathewi,Phra Nakhon,13.758889,100.534444,10400,1
21,"ป้อมปราบ,วัดเทพศิรินทร์,คลองมหานาค,บ้านบาตร,วั...",Nightclub,Music Venue,Whisky Bar,Strip Club,Speakeasy,Pom Prap Sattru Phai,Phra Nakhon,13.758056,100.513056,10100,1
23,"มหาพฤฒาราม,สีลม,สุริยวงศ์,บางรัก,สี่พระยา",Nightclub,Music Venue,Health & Beauty Service,Gay Bar,Cocktail Bar,Bang Rak,Phra Nakhon,13.730833,100.524167,10500,1
25,"ราษฎร์บูรณะ,บางปะกอก,บางมด,ทุ่งครุ",Nightclub,Gay Bar,Whisky Bar,Strip Club,Speakeasy,Rat Burana,Thorn Buri,13.682222,100.505556,10140,0
36,"สามเสนใน,ถนนเพชรบุรี,ทุ่งพญาไท,มักกะสัน,ถนนพญา...",Nightclub,Pub,Whisky Bar,Strip Club,Speakeasy,Phaya Thai,Phra Nakhon,13.780000,100.542778,10400,1
39,"ห้วยขวาง,บางกะปิ,ดินแดง,สามเสนนอก",Nightclub,Strip Club,Other Nightlife,Music Venue,Bar,Huai Khwang,Phra Nakhon,13.776667,100.579444,10310,1
